# SPSS ModelerのフラグノードをPythonで書き換える。購入商品カテゴリごとの集計


■加工前<BR>
誰（CUSTID）がいつ（SDATE）何（PRODUCTID、L_CLASS商品大分類、M_CLASS商品中分類）をいくら（SUBTOTAL）購入したかが記録されたID付POSデータを使います。

![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/4254f2e9-56d7-30ad-705b-9bf54f5a73d3.png)


In [1]:
!wget https://raw.githubusercontent.com/hkwd/200611Modeler2Python/master/data/sampletranDEPT4en2019S.csv

--2020-07-27 08:21:01--  https://raw.githubusercontent.com/hkwd/200611Modeler2Python/master/data/sampletranDEPT4en2019S.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1463149 (1.4M) [text/plain]
Saving to: ‘sampletranDEPT4en2019S.csv’

100%[======================================>] 1,463,149   --.-K/s   in 0.04s   

2020-07-27 08:21:01 (35.7 MB/s) - ‘sampletranDEPT4en2019S.csv’ saved [1463149/1463149]



In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

import pandas as pd
df = pd.read_csv('sampletranDEPT4en2019S.csv')

In [3]:
#pd.options.display.max_rows = 10
#print(pd.options.display.max_rows)

In [4]:
df.head(15)

,CUSTID,SDATE,PRODUCTID,L_CLASS,M_CLASS,SUBTOTAL
0,100001,2019-07-02,9937845,BAG,BAG01,1740
1,100001,2019-07-09,9903878,BAG,BAG01,3108
2,100001,2019-08-08,9922209,COSMETICS,COSMETICS03,2888
3,100001,2019-08-18,9901187,SHOES,SHOES02,2700
4,100001,2019-08-26,9910507,COSMETICS,COSMETICS03,3385
5,100001,2019-10-03,9903857,BAG,BAG01,2395
6,100001,2019-10-06,9905074,SHOES,SHOES02,1372
7,100001,2019-10-16,9937849,SHOES,SHOES01,5838
8,100001,2019-11-02,9937847,COSMETICS,COSMETICS02,4000
9,100001,2019-11-02,9937848,SHOES,SHOES03,15288


# ①各顧客毎に商品カテゴリ(M_CLASS)ごとの購入ありなしのフラグ化

![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/1f7893c7-f30a-6754-18fe-d501dc437b76.png)


データ型ノードの書き換え
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/e3148a4c-600a-176d-4e4b-b8555dce2b14.png)

pandasでカテゴリ変数をダミー変数に変換（get_dummies） | note.nkmk.me<BR>
https://note.nkmk.me/python-pandas-get-dummies/
<BR>【python】機械学習でpandas.get_dummiesを使ってはいけない - 静かなる名辞<BR>
https://www.haya-programming.com/entry/2019/08/17/184527
<BR>PandasのCategorical関係を調べてみた～慣れれば便利（と思う） - Qiita<BR>
https://qiita.com/TomokIshii/items/8acb138bd36e1b51b148

In [5]:
#カテゴリ変数をあらかじめ定義する
#テストデータやスコアリング時にパターンを持っていないデータでも同じフラグを持つようにするため
df1=df.copy()
M_CLASS_list=df1['M_CLASS'].unique()
df1["M_CLASS"] = pd.Categorical(df1["M_CLASS"], categories=M_CLASS_list)
df1["M_CLASS"].cat.categories

Index(['BAG01', 'COSMETICS03', 'SHOES02', 'SHOES01', 'COSMETICS02', 'SHOES03',
       'BAG03', 'BAG02', 'COSMETICS01'],
      dtype='object')

フラグノードの書き換え
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/f51b1172-2850-1236-fff9-b0a10c416d33.png)

In [6]:
#columnsでフラグ化する列を指定する
#prefixでフラグ化した列のPrefixを指定する。ここではModelerに倣ってフラグ化する列をいれている。
#一人一行にしたい場合はgroupbyする。あらかじめ列を絞って(df[['CUSTID','M_CLASS']])から集計するのがよい。
#１回でも購入したかどうかのフラグにするならばmaxをとる
#なお0,1ではなくT,Fにしたい場合はreplace(1,'T').replace(0,'F')をいれるが、0,1のほうが何かと使いやすい。
df_flg1=pd.get_dummies(df[['CUSTID','M_CLASS']],
                       columns=["M_CLASS"],
                       prefix="M_CLASS").groupby('CUSTID').max()
df_flg1.head()

,M_CLASS_BAG01,M_CLASS_BAG02,M_CLASS_BAG03,M_CLASS_COSMETICS01,M_CLASS_COSMETICS02,M_CLASS_COSMETICS03,M_CLASS_SHOES01,M_CLASS_SHOES02,M_CLASS_SHOES03
CUSTID,,,,,,,,,
100001,1,0,0,0,1,1,1,1,1
100004,0,0,1,0,1,1,0,0,1
100006,0,1,1,0,1,0,0,0,0
100008,0,0,1,1,1,1,0,0,1
100012,0,0,1,1,1,0,0,0,1


In [7]:
#カテゴリ定義していないdfだと列数が変わってしまう
pd.get_dummies(df[['CUSTID','M_CLASS']].head(5),prefix="M_CLASS",columns=["M_CLASS"]).groupby('CUSTID').max()

,M_CLASS_BAG01,M_CLASS_COSMETICS03,M_CLASS_SHOES02
CUSTID,,,
100001,1,1,1


In [8]:
#カテゴリ定義しているdf1では列数が変わらない
pd.get_dummies(df1[['CUSTID','M_CLASS']].head(5),prefix="M_CLASS",columns=["M_CLASS"]).groupby('CUSTID').max()

,M_CLASS_BAG01,M_CLASS_COSMETICS03,M_CLASS_SHOES02,M_CLASS_SHOES01,M_CLASS_COSMETICS02,M_CLASS_SHOES03,M_CLASS_BAG03,M_CLASS_BAG02,M_CLASS_COSMETICS01
CUSTID,,,,,,,,,
100001,1,1,1,0,0,0,0,0,0


### ②各顧客毎に商品カテゴリ(M_CLASS)ごとの購回数を集計

![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/b078d5bd-05be-640c-792a-da5af74c7f81.png)

![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/d6bdf8d2-40a8-7d48-b046-4179931993fc.png)
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/e2dcacce-b25e-0821-e422-b9bad781b652.png)

In [9]:
#columnsでフラグ化する列を指定する
#prefixでフラグ化した列のPrefixを指定する。ここではModelerに倣ってフラグ化する列をいれている。
#一人一行にしたい場合はgroupbyする。あらかじめ列を絞って(df[['CUSTID','M_CLASS']])から集計するのがよい。
#購入回数なのでsumをとる
df_sum=pd.get_dummies(df1[['CUSTID','M_CLASS']],
                       columns=["M_CLASS"],
                       prefix="M_CLASS").groupby('CUSTID').sum()
df_sum.head()

,M_CLASS_BAG01,M_CLASS_COSMETICS03,M_CLASS_SHOES02,M_CLASS_SHOES01,M_CLASS_COSMETICS02,M_CLASS_SHOES03,M_CLASS_BAG03,M_CLASS_BAG02,M_CLASS_COSMETICS01
CUSTID,,,,,,,,,
100001,3,2,3,1,1,1,0,0,0
100004,0,3,0,0,1,4,1,0,0
100006,0,0,0,0,1,0,1,1,0
100008,0,1,0,0,2,6,1,0,1
100012,0,0,0,0,1,4,2,0,1


### ③商品カテゴリ(M_CLASS)をつかったモデルを作成するためにダミー変数化

![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/e42edb55-1f4c-1029-3941-d984d5ddc030.png)
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/6c5ac922-d297-91e0-7e1a-b1c973f4326c.png)

pandas.get_dummies — pandas 1.0.5 documentation
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html

In [10]:
#フラグノードはget_dummiesで置き換えられる
#Modelerと異なり、フラグ化した列の元列はなくなってしまう。必要なケースは少ないが、必要な場合は再度JOINする必要がある。
df_dummy=pd.get_dummies(df1,columns=["M_CLASS"],prefix="M_CLASS",drop_first=True)
df_dummy.head()

,CUSTID,SDATE,PRODUCTID,L_CLASS,SUBTOTAL,M_CLASS_COSMETICS03,M_CLASS_SHOES02,M_CLASS_SHOES01,M_CLASS_COSMETICS02,M_CLASS_SHOES03,M_CLASS_BAG03,M_CLASS_BAG02,M_CLASS_COSMETICS01
0,100001,2019-07-02,9937845,BAG,1740,0,0,0,0,0,0,0,0
1,100001,2019-07-09,9903878,BAG,3108,0,0,0,0,0,0,0,0
2,100001,2019-08-08,9922209,COSMETICS,2888,1,0,0,0,0,0,0,0
3,100001,2019-08-18,9901187,SHOES,2700,0,1,0,0,0,0,0,0
4,100001,2019-08-26,9910507,COSMETICS,3385,1,0,0,0,0,0,0,0


# scikitlearnのOneHotEncoderを使うという方法もあります。

In [11]:
!pip install sklearn-pandas

Python: sklearn-pandas で scikit-learn と pandas の食べ合わせを改善する<BR>
https://blog.amedama.jp/entry/sklearn-pandas

In [12]:
df2=df.copy()
import numpy as np
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import OneHotEncoder
mapper = DataFrameMapper([
    (['CUSTID'],None),
    (['M_CLASS'], OneHotEncoder(dtype=np.uint8))],  df_out=True)
df2=mapper.fit_transform(df2).groupby('CUSTID').max()
df2.head()

,M_CLASS_x0_BAG01,M_CLASS_x0_BAG02,M_CLASS_x0_BAG03,M_CLASS_x0_COSMETICS01,M_CLASS_x0_COSMETICS02,M_CLASS_x0_COSMETICS03,M_CLASS_x0_SHOES01,M_CLASS_x0_SHOES02,M_CLASS_x0_SHOES03
CUSTID,,,,,,,,,
100001,1,0,0,0,1,1,1,1,1
100004,0,0,1,0,1,1,0,0,1
100006,0,1,1,0,1,0,0,0,0
100008,0,0,1,1,1,1,0,0,1
100012,0,0,1,1,1,0,0,0,1


In [13]:
import sys

print(sys.version)

3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) 
[GCC 7.3.0]


In [14]:
print(pd.__version__)

0.24.1
